# Inhheritance
In this notebook we show how to setup an inharitance plan with simplicity



# 1) Simple Inheritance

The inheritance strategy implemented here is a multi-path spending logic based on different roles:

- There is an inheritor (Alice) who can spend after a time delay (a timelock is enforced) using her signature.
- A cold key holder (Bob) can spend immediately with their signature, representing a high-security key usage.
- A hot key holder (Charlie) can spend immediately but only if the transaction recreates the same covenant structure, ensuring control is refreshed but the inheritance rules stay intact.

Overall, the strategy protects funds by allowing immediate spend via a cold key, conditional refresh spending via a hot key that enforces recursive constraints, and delayed inheritance spend by a designated inheritor. This design balances security, upgradability, and inheritance enforcement using cryptographic checks and time-locks.

```mermaid
flowchart TD
    Start[Start: Spending Decision]
    Start --> DecisionInherit{Is it an inheritance spend?}
    DecisionInherit -- Yes --> Inherit["Inherit Spend: Timelock 180 days<br>Verify inheritor signature (Alice)"]
    DecisionInherit -- No --> DecisionColdHot{Is it cold or hot spend?}
    DecisionColdHot -- Cold --> ColdSpend["Cold Spend: Verify cold signature (Bob)"]

    DecisionColdHot -- Hot --> HotSpend["Hot Spend: Verify hot signature (Charlie)<br>Enforce recursive covenant (outputs preserved)"]

    Inherit --> End[Spend Allowed]
    ColdSpend --> End
    HotSpend --> End


```


In [1]:
import pysimplicityhl
import pyhalsimplicity
import os
import json
import sys
sys.path.append("..")
import lib

basedir = os.getcwd()


In [2]:
from pathlib import Path

_temp_folder = Path("../_tmp")
_temp_folder.mkdir(parents=True, exist_ok=True)  # creates folder if not existing
lib.clear_folder(_temp_folder)


In [3]:
contract_code = """

fn checksig(pk: Pubkey, sig: Signature) {
    // Compute the standard signature message hash for this spending transaction.
    let msg: u256 = jet::sig_all_hash();
    // Verify that `sig` is a valid Schnorr signature (BIP340) by `pk` over `msg`.
    jet::bip_0340_verify((pk, msg), sig);
}

fn recursive_covenant() {
    // Enforce that the spending transaction has exactly 2 outputs.
    assert!(jet::eq_32(jet::num_outputs(), 2));
    // Get the hash of the currently executing script (this covenant script).
    let this_script_hash: u256 = jet::current_script_hash();
    // Read the script hash of output 0 and fail if it does not exist.
    let output_script_hash: u256 = unwrap(jet::output_script_hash(0));
    // Enforce that output 0 uses the same script as this input (recursive covenant).
    assert!(jet::eq_256(this_script_hash, output_script_hash));
    // Enforce that output 1 is a fee output to the miner.
    assert!(unwrap(jet::output_is_fee(1)));
}

fn inherit_spend(inheritor_pk: Pubkey, inheritor_sig: Signature) {
    // Require that a relative timelock of 180 days has passed before spending.
    let days_180: Distance = 25920;
    jet::check_lock_distance(days_180);
    // After the timelock, allow the inheritor to spend using their key/signature.
    checksig(inheritor_pk, inheritor_sig);
}

fn cold_spend(cold_pk: Pubkey, cold_sig: Signature) {
    // Immediate spend path for the cold (high‑security) key.
    checksig(cold_pk, cold_sig);
}

fn refresh_spend(hot_pk: Pubkey, hot_sig: Signature) {
    // Spend using a hot key (e.g. for refreshing or rotating the UTXO).
    checksig(hot_pk, hot_sig);
    // Require that the new output recreates the same covenant structure.
    recursive_covenant();
}


/*
This is the main entry point of the program that controls spending logic based on who is spending.

- It initializes public keys for three parties: Alice, Bob, and Charlie.
- Depending on the condition `INHERIT_OR_NOT` from the witness, it routes spending:
    - If an inheritor signature is provided (Left variant), it calls `inherit_spend` with Alice's key.
    - Otherwise (Right variant), it checks if the signature is from cold storage (Left) or hot storage (Right):
        - For cold storage (Bob's key), it calls `cold_spend`.
        - For hot storage (Charlie's key), it calls `refresh_spend`, which also enforces the recursive covenant.
*/
fn main() {
    let alice_pk: Pubkey = witness::ALICE_PUBLIC_KEY;
    let bob_pk: Pubkey = witness::BOB_PUBLIC_KEY;
    let charlie_pk: Pubkey = witness::CHARLIE_PUBLIC_KEY;
    
    match witness::INHERIT_OR_NOT {
        Left(inheritor_sig: Signature) => inherit_spend(alice_pk, inheritor_sig),
        Right(cold_or_hot: Either<Signature, Signature>) => match cold_or_hot {
            Left(cold_sig: Signature) => cold_spend(bob_pk, cold_sig),
            Right(hot_sig: Signature) => refresh_spend(charlie_pk, hot_sig),
        },
    }
}
"""
contract_witness = """
{
    "INHERIT_OR_NOT": {
        "value": "Left(0x755201bb62b0a8b8d18fd12fc02951ea3998ba42bfc6664daaf8a0d2298cad43cdc21358c7c82f37654275dc2fea8c858adbe97bac92828b498a5a237004db6f)",
        "type": "Either<Signature, Either<Signature, Signature>>"
    },
    "ALICE_PUBLIC_KEY": {
        "value": "0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798",
        "type": "u256"
    },
    "BOB_PUBLIC_KEY": {
        "value": "0xc6047f9441ed7d6d3045406e95c07cd85c778e4b8cef3ca7abac09b95c709ee5",
        "type": "u256"
    },
    "CHARLIE_PUBLIC_KEY": {
        "value": "0xf9308a019258c31049344f85f89d5229b531c845836f99b08601f113bce036f9",
        "type": "u256"
    }

}
"""

In [4]:
#data = lib.compile(contract_code, contract_witness, folder=_temp_folder, delete_temp_files=False)
data = lib.compile(contract_code, contract_witness, folder=_temp_folder)
[print(a) for a in data.keys()]
# locals().update(data)
print()


status
program
witness

